In [1]:
import os
import pandas as pd
import numpy as np
from pandas_plink import read_plink
from sklearn.linear_model import LinearRegression

In [ ]:
# position of SAMD11, this gene is randomly chosen by Yunpeng
# 1 424879 1443377 SAMD11 0 +

In [3]:
# read and clean
gene_annot_df = pd.read_csv('gene_annot.txt', sep='\t')

expression_df = pd.read_csv('GD462.GeneQuantRPKM.50FN.samplename.resk10.txt', sep='\t')

expression_df['TargetID_clean'] = expression_df['TargetID'].str.split('.').str[0]

In [4]:
expression_df

,TargetID,Gene_Symbol,Chr,Coord,HG00096,HG00097,HG00099,HG00100,HG00101,HG00102,...,NA20811,NA20812,NA20813,NA20814,NA20815,NA20816,NA20819,NA20826,NA20828,TargetID_clean
0,ENSG00000152931.6,ENSG00000152931.6,5,59783540,0.101858,0.078110,0.048981,0.118597,0.004035,0.010925,...,0.240010,0.137175,0.148494,0.038643,0.088509,0.029204,0.024423,0.044816,0.139186,ENSG00000152931
1,ENSG00000183696.9,ENSG00000183696.9,7,48128225,8.183805,5.686911,2.434653,3.830894,6.612288,4.709646,...,6.094500,12.536000,2.217262,3.573394,7.583364,4.052882,1.570378,4.900372,6.737308,ENSG00000183696
2,ENSG00000139269.2,ENSG00000139269.2,12,57846106,1.199910,1.573572,0.521616,1.447225,3.565791,1.982681,...,1.996067,2.854923,2.267343,1.331201,2.187895,1.004250,3.003316,1.984362,1.684954,ENSG00000139269
3,ENSG00000169129.8,ENSG00000169129.8,10,116164515,0.831940,0.069778,0.931086,0.620941,1.660668,0.570481,...,1.127852,0.774409,1.495854,0.895342,1.513521,0.826377,1.021201,0.952502,0.740565,ENSG00000169129
4,ENSG00000134602.11,ENSG00000134602.11,X,131157293,27.646422,24.395572,16.445374,24.806650,25.113349,19.233988,...,28.725528,24.450520,27.264069,26.912814,29.509210,26.462331,25.624009,25.707741,22.824957,ENSG00000134602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23717,ENSG00000235472.1,ENSG00000235472.1,13,29172970,31.582832,34.071123,19.394365,37.523721,33.430473,37.844966,...,33.224977,39.827675,27.096811,34.686342,37.386766,30.598840,33.516674,32.017940,38.341888,ENSG00000235472
23718,ENSG00000114423.14,ENSG00000114423.14,3,105588396,14.054749,14.477899,11.584425,12.637956,12.015089,13.750655,...,9.900372,10.473115,13.433413,15.832594,19.216176,10.213739,14.563192,15.637732,8.357117,ENSG00000114423
23719,ENSG00000243312.2,ENSG00000243312.2,4,87791344,1.112114,0.831797,0.253228,0.271568,0.486086,1.362640,...,0.645389,0.819469,0.515448,0.463054,1.580658,0.701396,0.771233,0.857330,0.703369,ENSG00000243312
23720,ENSG00000257337.1,ENSG00000257337.1,12,53448222,3.826396,6.045798,2.593872,4.447169,5.294657,4.106823,...,6.017795,3.460273,3.905032,5.023161,5.333027,5.694370,8.142939,5.622043,6.026476,ENSG00000257337


In [5]:
# SAMD11 expression
samd11_expression = expression_df.loc[expression_df['TargetID_clean'] == 'ENSG00000187634']

In [7]:
samd11_expression.shape

(1, 467)

In [ ]:
gene_annot_df

In [ ]:
from pandas_plink import read_plink

# read Plink extracted bed
(bim, fam, bed) = read_plink('1000G.EUR.1.SAMD11.bed')

X = bed.T.compute()  # 使用 bed 文件作为自变量 X

In [ ]:
# common sample
common_samples = fam_ids[fam_ids.isin(expression_sample_ids)]
# reorder
common_samples_idx = fam_ids[fam_ids.isin(common_samples)].index
X_reordered = X[common_samples_idx, :]  # 重新排序后的基因型数据

In [ ]:
samd11_expression = expression_df.loc[expression_df['TargetID_clean'] == 'ENSG00000187634', matched_ids]
y = samd11_expression.values.flatten() 

In [ ]:
import statsmodels.api as sm
import numpy as np

# intercept
X_with_const = sm.add_constant(X_reordered)

# lr
ols_model = sm.OLS(y, X_with_const)
results = ols_model.fit()

# p value
p_values = results.pvalues[1:] 

In [ ]:
bim_df = pd.read_csv('1000G.EUR.1.SAMD11.bim', sep='\t', header=None)

# 4th col gene position
snp_positions = bim_df[3].values 

In [ ]:
# code used to draw locuszoom in with plt
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 6))
# plt.scatter(snp_positions, -np.log10(p_values), c='blue', alpha=0.5)

# plt.xlabel('Genomic Coordinate (bp)')
# plt.ylabel('-log10(p-value)')
# plt.title('LocusZoom Plot for SAMD11')

# plt.show()

In [ ]:
# prepare and output text file to draw locuszoom graph on the locuszoom website

bim_df = pd.read_csv('1000G.EUR.1.SAMD11.bim', sep='\t', header=None)


bim_df.columns = ['CHR', 'SNP', 'cm', 'BP', 'A1', 'A2']
snp_positions = bim_df[['SNP', 'CHR', 'BP']]


p_values_df = pd.DataFrame({
    'SNP': bim_df['SNP'],
    'P': p_values,        # p value
    'BP': bim_df['BP'],   # gene position
    'CHR': bim_df['CHR']  # chrom number
})

# check
if len(p_values_df) != len(bim_df):
    raise ValueError("Not Match")

# save
p_values_df.to_csv('locuszoom_input.txt', sep='\t', index=False)